# Summary
Sketching to add MDs from processed spectra to 'normal' SpectrumDocuments from Spec2Vec.

Later I should add MDs to the object, but for now I just add words called md@...

No real info here yet, struggling with smart_open version

In [12]:
import os
import gensim
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle
from math import ceil

data_path = "/mnt/scratch/louwe015/Mass_differences/data/"

In [2]:
all_pos_version1 = False
if all_pos_version1:
    all_pos = "gnps_positive_ionmode_cleaned_by_matchms_and_lookups"
else:
    all_pos = "ALL_GNPS_210125_positive_cleaned_by_matchms_and_lookups"

print(all_pos)

ALL_GNPS_210125_positive_cleaned_by_matchms_and_lookups


In [17]:
if all_pos_version1:
    jacc_mat_file = None  # I didn't run this one
else:
    jacc_mat_file = "jaccard_matrix_allpos2_04-03-2021.npz"

jacc_mat_path = os.path.join(data_path, 'jaccard_matrices', jacc_mat_file)
print(jacc_mat_file, os.path.isfile(jacc_mat_path))

jaccard_matrix_allpos2_04-03-2021.npz True


## Load normal SpectrumDocuments and MD 'documents'
From notebook 1.

In [3]:
# load md documents
md_documents_file = os.path.join(data_path, all_pos + "_mass_difference_documents.pickle")
if os.path.exists(md_documents_file):
    with open(md_documents_file, 'rb') as inf:
        md_documents = pickle.load(inf)  # list of list of tuples 'documents'
    print(len(md_documents))
else:
    print("error")

115344


In [24]:
# load SpectrumDocuments from normally processed spectra
documents_file = os.path.join(data_path, all_pos + "_processed_documents.pickle")
if os.path.exists(documents_file):
    with open(documents_file, 'rb') as inf:
        documents_spectrums_processed = pickle.load(inf)
    print(len(documents_spectrums_processed))
else:
    print('error')

AttributeError: module 'smart_open' has no attribute 'local_file'

## See if I can add MD as a new word to existing document
For now I will calculate the new intensity as: sqrt(count) * max_intensity with a maximum of 1.

In [18]:
# Load 'white listed' MDs from notebook 4
out_indices = os.path.join(data_path, 'jaccard_matrices', jacc_mat_file.rpartition('.')[0]+"_chosen_MDs_0.1-0.5.txt")
print(out_indices, os.path.isfile(out_indices))
with open(out_indices, 'r') as inf:
    chosen_mds = [line.strip() for line in inf]
len(chosen_mds), chosen_mds[:5]

/mnt/scratch/louwe015/Mass_differences/data/jaccard_matrices/jaccard_matrix_allpos2_04-03-2021_chosen_MDs_0.1-0.5.txt True


(16595, ['37.00', '37.01', '37.02', '37.05', '37.06'])

In [22]:
test_doc_md = md_documents[0]
set_chosen_mds = set(chosen_mds)
test_doc_chosen_mds = [tup for tup in test_doc_md if tup[0] in set_chosen_mds]
print(len(test_doc_md), len(test_doc_chosen_mds))
test_doc_md[:5], test_doc_chosen_mds[:5]

100 27


([('43.32', [0.6402318831480545], 1),
  ('43.97', [0.5162807397553768], 1),
  ('45.13', [0.26421245665598025], 1),
  ('45.24', [0.5498513452178678], 1),
  ('54.26', [0.33073825117099154], 1)],
 [('58.04', [0.32066137970996295], 1),
  ('79.98', [0.27045556510399293], 1),
  ('80.97', [0.25036983844828736], 1),
  ('85.02', [0.40764241232768667], 1),
  ('101.03', [0.537696707620307], 1)])

In [27]:
for md_tup in test_doc_chosen_mds:
    word = f"md@{md_tup[0]}"
    intensity = np.sqrt(md_tup[2])*max(md_tup[1])
    if intensity > 1:
        intensity = 1.0
    print(word, intensity)
    break

md@58.04 0.32066137970996295
